In [1]:
!pip install nltk
!pip install gensim
!pip install itertools
!pip install spacy
!pip install langdetect
!pip install pprint
!pip install pyLDAvis

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=8715c4ab97f3358fe55c7da3d1234622ba819e502ef0668e28ef34a51aee63c0
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.7 MB/s eta 0:00:00


In [9]:
import re
import pandas as pd
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import string

from collections import Counter
from nltk.stem import WordNetLemmatizer

#  GENSIM
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.corpora import MmCorpus
# for Defaultdics
from collections import defaultdict
# itertools
import itertools
# TF-IDF
from gensim.models.tfidfmodel import TfidfModel

# spacy for lemmatization
import spacy

# to delete non english text
from langdetect import detect, DetectorFactory

from gensim.models import CoherenceModel

from string import punctuation

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline


import gensim.models

from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from sklearn import decomposition
from scipy import linalg

from sklearn.feature_extraction.text import TfidfVectorizer

import collections
from nltk.util import ngrams

from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [10]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('omw-1.4')  # Optional for better language support
nltk.download('averaged_perceptron_tagger')  # For POS tagging
nltk.download('averaged_perceptron_tagger_eng')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading packa

True

In [11]:
from google.colab import files
uploaded = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving top5banksReviews_v1.csv to top5banksReviews_v1.csv


In [12]:
import io

df = pd.read_csv(io.BytesIO(uploaded["top5banksReviews_v1.csv"]))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
df_copy = df.copy()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11980 entries, 0 to 11979
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  11980 non-null  int64 
 1   rating      11980 non-null  int64 
 2   title       11979 non-null  object
 3   review      11980 non-null  object
 4   Bank        11980 non-null  object
dtypes: int64(2), object(3)
memory usage: 468.1+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
df_copy.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,rating,title,review,Bank
0,0,4,Could Use Improvements,"If there is only one possible account, I shoul...",BMO
1,1,4,Verify,Suddenly I am being asked to verify this devic...,BMO
2,2,1,Works terrible on my new phone,"On my new phone, practically every time I open...",BMO
3,3,1,Bare bones app,No features to make this app stand out amongst...,BMO
4,4,1,Not working with iOS12.2,This app won’t work with iOS12.2. Every time I...,BMO


In [14]:
# CHAT GPT CODE

DetectorFactory.seed = 0

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

df_copy['is_english'] = df_copy['review'].apply(is_english)
df = df_copy[df_copy['is_english']].drop(columns=['is_english'])



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11118 entries, 0 to 11979
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  11118 non-null  int64 
 1   rating      11118 non-null  int64 
 2   title       11117 non-null  object
 3   review      11118 non-null  object
 4   Bank        11118 non-null  object
dtypes: int64(2), object(3)
memory usage: 521.2+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
longest_length = df['review'].str.len().max()
shortest_length = df['review'].str.len().min()
average_length = df['review'].str.len().mean()

print("The longest review is {} characters long while the shortest is {} characters long and on average, the reviews are {} characters.".format(longest_length, shortest_length, average_length))

The longest review is 2926 characters long while the shortest is 4 characters long and on average, the reviews are 189.7167656053247 characters.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Data Cleaning:
- Remove stopwords
- Remove punctuation
- Lower case everything
- Stemming and Lemming

In [15]:
lemmer = nltk.WordNetLemmatizer()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
punctuation_to_remove = r"[\[\]\(\)!?'|\.,;:]"

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'app', 'you', 'my',
                   'this', 'not'])

stemmer = PorterStemmer()

def remove_punc(sent):
  no_punc = re.sub(punctuation_to_remove, "", sent)
  return no_punc


def clean_text(text):
  text = ' '.join([word for word in text.split() if word not in stop_words])
  text = ''.join([word for word in text if word not in string.punctuation])
  text = ''.join([word for word in text if word not in punctuation_to_remove])
  text = text.lower()

  return text

def lemmize(text):
  text = [lemmer(word) for word in text.split()]

  return str(text)

# def stemmize(text):
#   text = [stemmer.stem(word) for word in text.split()]
#         #[[stemmer.stem(word) for word in sentence] for sentence in tokenize_doc_to_words]
#   return str(text)


def stemmizeLEMMER(text):
    if not isinstance(text, str):
        raise TypeError("Input text must be a string.")
    #lemmer
    text = [lemmer.lemmatize(word) for word in text.split()]
    #text = [stemmer.stem(word) for word in text.split()]
    return text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
df['noStopWords'] = df['review'].apply(lambda x: clean_text(str(x.strip())))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
df['Stemm_'] = df['noStopWords'].apply(lambda x: stemmizeLEMMER(x))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,rating,title,review,Bank,noStopWords,Stemm_
0,0,4,Could Use Improvements,"If there is only one possible account, I shoul...",BMO,if one possible account i select time i need d...,"[if, one, possible, account, i, select, time, ..."
1,1,4,Verify,Suddenly I am being asked to verify this devic...,BMO,suddenly i asked verify device almost every ti...,"[suddenly, i, asked, verify, device, almost, e..."
2,2,1,Works terrible on my new phone,"On my new phone, practically every time I open...",BMO,on new phone practically every time i open ask...,"[on, new, phone, practically, every, time, i, ..."
3,3,1,Bare bones app,No features to make this app stand out amongst...,BMO,no features make stand amongst canadian banks ...,"[no, feature, make, stand, amongst, canadian, ..."
4,4,1,Not working with iOS12.2,This app won’t work with iOS12.2. Every time I...,BMO,this won’t work ios122 every time i open turns...,"[this, won’t, work, ios122, every, time, i, op..."


In [47]:
#df['CLEAN_review'] = df['Stemm'].apply(lambda x: lemmize(x))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


To have eaasy access - we save the data in several formats:
- reviews will include the RAW original data
- reviews without the stop words
- reviews without the stop words + stemming

Then each of these is:
1. Tokenized to sentence, named "TOK_SENT"
2. Tokenized to words, named "TOK_WORD"
3. Cleaned


In [179]:
#' '.join(reviews_stemmed[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'if one possible account i select time i need deposit cheque if multiple account would good allowed choose default account deposit to other bank apps set way'

In [186]:
#reviews_stemmed[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['if',
 'one',
 'possible',
 'account',
 'i',
 'select',
 'time',
 'i',
 'need',
 'deposit',
 'cheque',
 'if',
 'multiple',
 'account',
 'would',
 'good',
 'allowed',
 'choose',
 'default',
 'account',
 'deposit',
 'to',
 'other',
 'bank',
 'apps',
 'set',
 'way']

In [184]:
#reviews_nostopwords[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'if one possible account i select time i need deposit cheque if multiple accounts would good allowed choose default account deposit to other bank apps set way'

In [20]:
reviews = list(df['review'])
reviews_nostopwords = list(df['noStopWords'])
reviews_stemmed = list(df['Stemm_'])


#tokenize to sentence
TOK_SENT_reviews = sent_tokenize(" ".join(reviews))

#tokenize to words
TOK_WORD_reviews = [word_tokenize(sentence) for sentence in TOK_SENT_reviews]
TOK_WORD_reviews_nostopwords = [word_tokenize(sentence) for sentence in reviews_nostopwords]
TOK_WORD_reviews_stemmed = reviews_stemmed

#cleaned sentences
TOK_SENT_reviews = [remove_punc(sentence) for sentence in TOK_SENT_reviews]

#cleaned words
for i in range(len(TOK_WORD_reviews)):
  TOK_WORD_reviews[i] = [w for w in TOK_WORD_reviews[i] if w.isalpha() and w not in punctuation and w not in stop_words]

for i in range(len(TOK_WORD_reviews_nostopwords)):
  TOK_WORD_reviews_nostopwords[i] = [w for w in TOK_WORD_reviews_nostopwords[i] if w.isalpha() and w not in punctuation and w not in stop_words]

for i in range(len(TOK_WORD_reviews_stemmed)):
  TOK_WORD_reviews_stemmed[i] = [w for w in TOK_WORD_reviews_stemmed[i] if w.isalpha() and w not in punctuation and w not in stop_words]

#gensim ready data
# gensim_ok_reviews_data = " ".join(reviews)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'matcher'])

gensim_ok_reviews_data = " ".join(reviews)

# FIND THE NER's : Name Entity  - CHAT GPT CODE - Because of run time
nlp.max_length = 10**7

# Split the large text into smaller chunks (e.g., 1 million characters each) - 1 million characters per chunk
chunk_size = 10**6
chunks = [gensim_ok_reviews_data[i:i+chunk_size] for i in range(0, len(gensim_ok_reviews_data), chunk_size)]

for chunk in chunks:
    doc_nlp = nlp(chunk)
    for ent in doc_nlp.ents[:10]:
        print(ent.label_, ent.text)


In [21]:
def dict_gesim_stuff(text):
  dictionary = dictionary = Dictionary(text)

  corpus = [dictionary.doc2bow(review) for review in text]

  # Save the corpus in Matrix Market format
  MmCorpus.serialize('mmcorpus_ios.mm', corpus)

  return dictionary, corpus


def print_top_50_words(corpus):
  total_word_count = defaultdict(int)
  word_weights = defaultdict(float)

  for word_id, word_count in itertools.chain.from_iterable(corpus):
      total_word_count[word_id] += word_count

  # sort this dictionary
  sorted_tota_words_count = sorted(total_word_count.items(), key = lambda w: w[1], reverse = True)

  tfidf = TfidfModel(corpus)


  # CHATGPT CODE:
  for doc in corpus:
      tfidf_weights = tfidf[doc]  # Calculate TF-IDF for the review
      for term_id, weight in tfidf_weights:
          word_weights[term_id] += weight  # Aggregate the weight for the term

  # Sort the terms by weight in descending order
  sorted_word_weights = sorted(word_weights.items(), key=lambda x: x[1], reverse=True)

  # Print the top 50 terms with their weights
  top_50_words = [(dictionary.get(term_id), weight) for term_id, weight in sorted_word_weights[:50]]

  for word, weight in top_50_words:
      print(word, weight)




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [189]:
df['Stemm_'][0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['if',
 'one',
 'possible',
 'account',
 'i',
 'select',
 'time',
 'i',
 'need',
 'deposit',
 'cheque',
 'if',
 'multiple',
 'account',
 'would',
 'good',
 'allowed',
 'choose',
 'default',
 'account',
 'deposit',
 'to',
 'other',
 'bank',
 'apps',
 'set',
 'way']

In [22]:
df['noStopWords'][0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'if one possible account i select time i need deposit cheque if multiple accounts would good allowed choose default account deposit to other bank apps set way'

In [23]:
# DICTIONARY GENSIM
# tokenized_doc_to_words_noStop is the cleaned version of the data
stemm_data = list(df['Stemm_'])

dictionary = Dictionary(stemm_data)

mmcorpus = [dictionary.doc2bow(review) for review in stemm_data] # each review is turned into bag of words

# Save the corpus in Matrix Market format
MmCorpus.serialize('mmcorpus_ios.mm', mmcorpus)

# Later, you can load it back without reading the whole file into memory
loaded_corpus = MmCorpus('mmcorpus_ios.mm')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [192]:
print_top_50_words(mmcorpus)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


i 369.65890700872734
app 243.2507929536397
easy 225.93491642069182
update 221.3450872462168
it 207.49544262990386
account 195.5598749534573
time 193.8293833370206
work 189.17911712010195
great 187.86003439448984
the 186.34754816081522
bank 185.59511063740877
banking 171.36416847536762
please 167.6916537516206
this 167.0959175823785
it’s 155.61690776984778
new 149.05561957608532
fix 143.08068037373678
need 142.04576981756222
can’t 137.03154920708053
love 136.5752892661613
like 130.03293897713664
get 128.55200974704377
good 122.73607003732471
card 121.81982232976121
issue 117.36591318725142
deposit 115.90752596591378
phone 115.08491190739932
password 114.76435160818144
log 114.73371393856017
version 113.3596129529998
make 112.81263027735113
one 112.1147663842888
cibc 111.84380879229475
open 111.37236332449744
service 110.88170858848373
every 110.6855646189885
even 110.28050701174534
would 110.0466247530332
credit 104.28455920605137
cheque 104.10974362904206
very 102.26978939236318
see 10

In [193]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,rating,title,review,Bank,noStopWords,Stemm_
0,0,4,Could Use Improvements,"If there is only one possible account, I shoul...",BMO,if one possible account i select time i need d...,"[if, one, possible, account, i, select, time, ..."
1,1,4,Verify,Suddenly I am being asked to verify this devic...,BMO,suddenly i asked verify device almost every ti...,"[suddenly, i, asked, verify, device, almost, e..."
2,2,1,Works terrible on my new phone,"On my new phone, practically every time I open...",BMO,on new phone practically every time i open ask...,"[on, new, phone, practically, every, time, i, ..."
3,3,1,Bare bones app,No features to make this app stand out amongst...,BMO,no features make stand amongst canadian banks ...,"[no, feature, make, stand, amongst, canadian, ..."
4,4,1,Not working with iOS12.2,This app won’t work with iOS12.2. Every time I...,BMO,this won’t work ios122 every time i open turns...,"[this, won’t, work, ios122, every, time, i, op..."


In [24]:
tokens = stemm_data #[[word.lower() for word in word_tokenize(sentence) if word.isalpha() and word not in punctuation and word not in stop_words]
    #for sentence in TOK_SENT_reviews_nostopwords]

# tokens = df['noStopWords']

unigrams = [list(ngrams(sentence, 1)) for sentence in tokens]  # Unigrams
bigrams = [list(ngrams(sentence, 2)) for sentence in tokens]  # Bigrams
trigrams = [list(ngrams(sentence, 3)) for sentence in tokens]  # Trigrams

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [196]:
bigrams[:1][0][0:3]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('if', 'one'), ('one', 'possible'), ('possible', 'account')]

In [25]:
def build_lda_model(n_topic, corpus, dictionary):
  lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                              num_topics = n_topic,
                                              id2word = dictionary,
                                              random_state = 100,
                                              update_every = 1,
                                              chunksize = 1000,
                                              passes = 10,
                                              alpha = 'auto',
                                              per_word_topics = True)
  return lda_model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
single_word_lda_model = []
topic_numbers_to_try = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

for i in range(len(topic_numbers_to_try)):
  single_word_lda_model.append(build_lda_model(topic_numbers_to_try[i], mmcorpus, dictionary))


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
def calculate_preplexity(model, corpus):
  preplexity = model.log_perplexity(corpus)
  return preplexity


def calculate_coherence(model, text, dictionary):
  coherence_model_lda = CoherenceModel(model = model, texts = text, dictionary = dictionary, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()

  return coherence_lda

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
models_preplexity = []
models_coherence = []
models_topics = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
tracker = 5

for model in single_word_lda_model:
  models_preplexity.append(calculate_preplexity(model, mmcorpus))
  models_coherence.append(calculate_coherence(model, stemm_data, dictionary))
  tracker += 1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
models_df = pd.DataFrame({
    'Num_Topics': models_topics,
    'Preplexity': models_preplexity,
    'Coherence': models_coherence
})

print(models_df)

    Num_Topics  Preplexity  Coherence
0            5   -7.138061   0.558985
1            6   -7.168225   0.490389
2            7   -7.209733   0.529730
3            8   -7.261791   0.537482
4            9   -7.346846   0.537647
5           10   -7.460067   0.498814
6           11   -7.637953   0.454285
7           12   -7.837432   0.502164
8           13   -8.047944   0.500220
9           14   -8.194538   0.453476
10          15   -8.305494   0.411846


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
print("Number of topics with the maximum Coherence is ", models_df.loc[models_df['Coherence'].idxmax(), 'Num_Topics'])

Number of topics with the maximum Coherence is  5


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
print("Number of topics with the minimum Preplexity is ", models_df.loc[models_df['Preplexity'].idxmin(), 'Num_Topics'])

Number of topics with the minimum Preplexity is  15


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Because we get different numbers of topics for this, I'll use a combination of both scores to pick the best number of topics!

- Normalize Preplexity and Coherence (MinMaxScaler)
- Combine the scores (weight of Preplexity and Coherence are assumed to be equal = 0.5)

score = w_1 * Coherence + w_2 (1 - Preplexity)
- The row with the highest score is the best.  


In [34]:
models_df['Normalized_Perplexity'] = (models_df['Preplexity'] - models_df['Preplexity'].min()) / (models_df['Preplexity'].max() - models_df['Preplexity'].min())
models_df['Normalized_Coherence'] = (models_df['Coherence'] - models_df['Coherence'].min()) / (models_df['Coherence'].max() - models_df['Coherence'].min())

models_df['Inverted_Perplexity'] = 1 - models_df['Normalized_Perplexity'] # because smaller is better

weight_preplexity = 0.5
weight_coherence = 0.5

models_df['Score'] = weight_coherence * models_df['Normalized_Coherence'] + weight_preplexity * models_df['Inverted_Perplexity']

best_row = models_df.loc[models_df['Score'].idxmax()]
print("Best Row:")
print(best_row)

Best Row:
Num_Topics               13.000000
Preplexity               -8.047944
Coherence                 0.500220
Normalized_Perplexity     0.220612
Normalized_Coherence      0.600612
Inverted_Perplexity       0.779388
Score                     0.690000
Name: 8, dtype: float64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
best_model_single_word = single_word_lda_model[8]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
def print_topics(model, corpus):
  pprint(model.print_topics())
  doc_lda = model[corpus]
  return doc_lda

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
print_topics(best_model_single_word, mmcorpus)


[(0,
  '0.044*"canada" + 0.043*"stock" + 0.043*"investing" + 0.037*"canadian" + '
  '0.036*"direct" + 0.034*"functionality" + 0.031*"far" + 0.025*"world" + '
  '0.018*"note" + 0.015*"space"'),
 (1,
  '0.052*"account" + 0.034*"card" + 0.032*"would" + 0.029*"credit" + '
  '0.026*"transaction" + 0.023*"see" + 0.022*"balance" + 0.021*"transfer" + '
  '0.020*"pay" + 0.019*"money"'),
 (2,
  '0.221*"deposit" + 0.166*"cheque" + 0.046*"photo" + 0.022*"recognition" + '
  '0.022*"picture" + 0.021*"taking" + 0.020*"trip" + 0.019*"camera" + '
  '0.019*"excellent" + 0.018*"hasn’t"'),
 (3,
  '0.096*"i" + 0.023*"app" + 0.021*"time" + 0.017*"it" + 0.015*"work" + '
  '0.012*"the" + 0.011*"get" + 0.011*"it’s" + 0.011*"this" + 0.010*"update"'),
 (4,
  '0.178*"very" + 0.064*"technical" + 0.035*"reinstall" + 0.034*"issue" + '
  '0.025*"live" + 0.019*"uninstall" + 0.018*"category" + 0.015*"complain" + '
  '0.014*"signed" + 0.014*"usd"'),
 (5,
  '0.035*"finally" + 0.033*"soon" + 0.031*"faceid" + 0.026*"report

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


For Bigrams, we will:

1.   Build several LDA models for Topic Modeling
2.   Start with number of topics of 10 (num_topics = 10), and try any value from 5 to 15
3.   The LDA model with the lowest Preplexity and best (higher) Coherence score is chosen.
4.   The words in topics are extracted and we decide names for each topic.
5.   Topic modeling is done for bigrams.



In [ ]:
# cleaned_sentences = []

# for sentence in tokenized_doc_to_sentences:
#   words = re.sub(r'[^\w\s]', '', sentence).lower().split() #remove punctuation, lower case and split into words
#   stemmed_words = [stemmer.stem(word) for word in words if word not in stop_words]
#   cleaned_sentences.append(' '.join(stemmed_words))


In [38]:
#CHATGPT
bigrams_combined = [[f"{a} {b}" for a, b in sublist] for sublist in bigrams]
bigrams_combined[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['if one',
 'one possible',
 'possible account',
 'account i',
 'i select',
 'select time',
 'time i',
 'i need',
 'need deposit',
 'deposit cheque',
 'cheque if',
 'if multiple',
 'multiple account',
 'account would',
 'would good',
 'good allowed',
 'allowed choose',
 'choose default',
 'default account',
 'account deposit',
 'deposit to',
 'to other',
 'other bank',
 'bank apps',
 'apps set',
 'set way']

In [39]:
dictionary_bigrams = Dictionary(bigrams_combined)
corpus_bigrams = [dictionary_bigrams.doc2bow(text) for text in bigrams_combined]
MmCorpus.serialize('mmcorpus_ios_bigrams.mm', corpus_bigrams)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
bigram_lda_model = []

for i in range(len(topic_numbers_to_try)):
  bigram_lda_model.append(build_lda_model(topic_numbers_to_try[i], corpus_bigrams, dictionary_bigrams))

models_preplexity_bigrams = []
models_coherence_bigrams = []

tracker = 5
for model in bigram_lda_model:
  models_preplexity_bigrams.append(calculate_preplexity(model, corpus_bigrams))
  models_coherence_bigrams.append(calculate_coherence(model, bigrams_combined, dictionary_bigrams))
  tracker += 1

models_df_bigrams = pd.DataFrame({
    'Num_Topics': models_topics,
    'Preplexity': models_preplexity_bigrams,
    'Coherence': models_coherence_bigrams
})

print("Bigrams --- models and scores: ")
print(models_df_bigrams)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Bigrams --- models and scores: 
    Num_Topics  Preplexity  Coherence
0            5  -12.235227   0.414372
1            6  -12.513615   0.374718
2            7  -13.403579   0.532268
3            8  -14.364463   0.532172
4            9  -15.297954   0.567798
5           10  -16.354079   0.565841
6           11  -17.405994   0.579607
7           12  -18.446354   0.571804
8           13  -19.404905   0.559364
9           14  -20.182787   0.552921
10          15  -20.944962   0.577760


In [41]:
print("Number of topics with the maximum Coherence is ", models_df_bigrams.loc[models_df_bigrams['Coherence'].idxmax(), 'Num_Topics'])

print("Number of topics with the minimum Preplexity is ", models_df_bigrams.loc[models_df_bigrams['Preplexity'].idxmin(), 'Num_Topics'])

Number of topics with the maximum Coherence is  11
Number of topics with the minimum Preplexity is  15


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
models_df_bigrams['Normalized_Perplexity'] = (models_df_bigrams['Preplexity'] - models_df_bigrams['Preplexity'].min()) / (models_df_bigrams['Preplexity'].max() - models_df_bigrams['Preplexity'].min())
models_df_bigrams['Normalized_Coherence'] = (models_df_bigrams['Coherence'] - models_df_bigrams['Coherence'].min()) / (models_df_bigrams['Coherence'].max() - models_df_bigrams['Coherence'].min())

models_df_bigrams['Inverted_Perplexity'] = 1 - models_df_bigrams['Normalized_Perplexity'] # because smaller is better

weight_preplexity = 0.5
weight_coherence = 0.5

models_df_bigrams['Score'] = weight_coherence * models_df_bigrams['Normalized_Coherence'] + weight_preplexity * models_df_bigrams['Inverted_Perplexity']

best_row_bigrams = models_df_bigrams.loc[models_df_bigrams['Score'].idxmax()]
print("Best Row:")
print(best_row_bigrams)

Best Row:
Num_Topics               15.000000
Preplexity              -20.944962
Coherence                 0.577760
Normalized_Perplexity     0.000000
Normalized_Coherence      0.990985
Inverted_Perplexity       1.000000
Score                     0.995493
Name: 10, dtype: float64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
best_model_bigrams = bigram_lda_model[-1]
print_topics(best_model_bigrams, corpus_bigrams)


[(0,
  '0.008*"mobile app" + 0.007*"new phone" + 0.007*"so i" + 0.007*"phone '
  'number" + 0.007*"love it" + 0.006*"number i" + 0.006*"everything i" + '
  '0.005*"don’t know" + 0.005*"make sure" + 0.004*"i iphone"'),
 (1,
  '0.005*"i issue" + 0.005*"go branch" + 0.005*"business account" + 0.004*"i '
  'found" + 0.004*"notification i" + 0.004*"i trying" + 0.004*"iphone 6" + '
  '0.004*"it’s time" + 0.004*"keep asking" + 0.004*"one account"'),
 (2,
  '0.017*"user friendly" + 0.012*"would like" + 0.009*"banking app" + '
  '0.008*"banking apps" + 0.006*"like see" + 0.006*"i know" + 0.006*"really '
  'good" + 0.006*"it’s easy" + 0.005*"home screen" + 0.005*"i changed"'),
 (3,
  '0.015*"last update" + 0.011*"since last" + 0.010*"work well" + 0.007*"thing '
  'i" + 0.006*"ever since" + 0.005*"would great" + 0.005*"right away" + '
  '0.004*"money account" + 0.004*"bank it’s" + 0.004*"the td"'),
 (4,
  '0.023*"credit card" + 0.020*"time i" + 0.020*"i can’t" + 0.018*"i don’t" + '
  '0.017*"plea

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
trigrams_combined = [[f"{a} {b} {c}" for a, b, c in sublist] for sublist in trigrams]
trigrams_combined[:3]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['if one possible',
  'one possible account',
  'possible account i',
  'account i select',
  'i select time',
  'select time i',
  'time i need',
  'i need deposit',
  'need deposit cheque',
  'deposit cheque if',
  'cheque if multiple',
  'if multiple account',
  'multiple account would',
  'account would good',
  'would good allowed',
  'good allowed choose',
  'allowed choose default',
  'choose default account',
  'default account deposit',
  'account deposit to',
  'deposit to other',
  'to other bank',
  'other bank apps',
  'bank apps set',
  'apps set way'],
 ['suddenly i asked',
  'i asked verify',
  'asked verify device',
  'verify device almost',
  'device almost every',
  'almost every time',
  'every time i',
  'time i try',
  'i try log',
  'try log in',
  'log in yes',
  'in yes i',
  'yes i enable',
  'i enable ‘trust',
  'enable ‘trust device’',
  '‘trust device’ every',
  'device’ every time',
  'every time what’s',
  'time what’s this'],
 ['on new phone',
  'new ph

In [45]:
dictionary_trigrams = Dictionary(trigrams_combined)
corpus_trigrams = [dictionary_trigrams.doc2bow(text) for text in trigrams_combined]
MmCorpus.serialize('mmcorpus_ios_trigrams.mm', corpus_trigrams)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
trigram_lda_model = []

for i in range(len(topic_numbers_to_try)):
  trigram_lda_model.append(build_lda_model(topic_numbers_to_try[i], corpus_trigrams, dictionary_trigrams))

models_preplexity_trigrams = []
models_coherence_trigrams = []

tracker = 5
for model in trigram_lda_model:
  models_preplexity_trigrams.append(calculate_preplexity(model, corpus_trigrams))
  models_coherence_trigrams.append(calculate_coherence(model, trigrams_combined, dictionary_trigrams))
  tracker += 1

models_df_trigrams = pd.DataFrame({
    'Num_Topics': models_topics,
    'Preplexity': models_preplexity_trigrams,
    'Coherence': models_coherence_trigrams
})

print("Trigrams --- models and scores: ")
print(models_df_trigrams)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Trigrams --- models and scores: 
    Num_Topics  Preplexity  Coherence
0            5  -13.497051   0.610250
1            6  -14.202739   0.644110
2            7  -15.335027   0.620675
3            8  -16.531823   0.499044
4            9  -19.365943   0.573855
5           10  -21.383319   0.569700
6           11  -24.083507   0.547007
7           12  -24.720050   0.663393
8           13  -25.936380   0.676689
9           14  -27.473640   0.653684
10          15  -27.826567   0.666610


In [47]:
print("Number of topics with the maximum Coherence is ", models_df_trigrams.loc[models_df_trigrams['Coherence'].idxmax(), 'Num_Topics'])

print("Number of topics with the minimum Preplexity is ", models_df_trigrams.loc[models_df_trigrams['Preplexity'].idxmin(), 'Num_Topics'])

Number of topics with the maximum Coherence is  13
Number of topics with the minimum Preplexity is  15


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
models_df_trigrams['Normalized_Perplexity'] = (models_df_trigrams['Preplexity'] - models_df_trigrams['Preplexity'].min()) / (models_df_trigrams['Preplexity'].max() - models_df_trigrams['Preplexity'].min())
models_df_trigrams['Normalized_Coherence'] = (models_df_trigrams['Coherence'] - models_df_trigrams['Coherence'].min()) / (models_df_trigrams['Coherence'].max() - models_df_trigrams['Coherence'].min())

models_df_trigrams['Inverted_Perplexity'] = 1 - models_df_trigrams['Normalized_Perplexity']

models_df_trigrams['Score'] = weight_coherence * models_df_trigrams['Normalized_Coherence'] + weight_preplexity * models_df_trigrams['Inverted_Perplexity']

best_row_trigrams = models_df_trigrams.loc[models_df_trigrams['Score'].idxmax()]
print("Best Row:")
print(best_row_trigrams)




Best Row:
Num_Topics               15.000000
Preplexity              -27.826567
Coherence                 0.666610
Normalized_Perplexity     0.000000
Normalized_Coherence      0.943263
Inverted_Perplexity       1.000000
Score                     0.971631
Name: 10, dtype: float64


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
best_model_trigrams = trigram_lda_model[-1]

print_topics(best_model_trigrams, corpus_trigrams)

[(0,
  '0.002*"other banking apps" + 0.001*"able deposit cheque" + 0.001*"email '
  'money transfer" + 0.001*"ever since i" + 0.001*"after recent update" + '
  '0.001*"after last update" + 0.001*"the recent update" + 0.000*"since i got" '
  '+ 0.000*"star i would" + 0.000*"look like it’s"'),
 (1,
  '0.003*"i can’t access" + 0.002*"please fix it" + 0.001*"access account i" + '
  '0.001*"called customer service" + 0.001*"i called customer" + 0.001*"why '
  'can’t i" + 0.001*"can’t access account" + 0.001*"app please fix" + '
  '0.001*"mobile banking app" + 0.001*"please fix app"'),
 (2,
  '0.003*"i love app" + 0.002*"please fix issue" + 0.001*"i can’t see" + '
  '0.001*"face id i" + 0.001*"i must say" + 0.001*"account i would" + '
  '0.001*"pretty much everything" + 0.001*"take photo cheque" + 0.001*"star i '
  'could" + 0.001*"it seems like"'),
 (3,
  '0.004*"everything i need" + 0.002*"the last update" + 0.002*"lump sum '
  'payment" + 0.002*"it if i" + 0.002*"i keep getting" + 0.001*"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best models:

- Single Words: (15 topics)
p = -8.047944 ------ c = 0.500220
- Bigrams:
p = -20.944962 ------ c = 0.577760
- Trigrams:
p =  -27.826567 ------ c = 0.666610

Final Best:
Trigrams

In [50]:
best_top_model_overall = best_model_trigrams

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:

pyLDAvis.enable_notebook()

vis = gensimvis.prepare(best_top_model_overall, corpus_trigrams, dictionary_trigrams)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.029424 -0.000185       1        1  14.092887
8      0.005727 -0.016140       2        1   9.593238
13     0.005633  0.010717       3        1   8.917103
6      0.005378  0.004464       4        1   7.470830
4      0.002870  0.000365       5        1   7.318961
9      0.002630  0.000308       6        1   7.059306
3      0.002164  0.000220       7        1   5.946520
10     0.001407  0.000112       8        1   5.721367
2      0.000859  0.000052       9        1   5.367537
14     0.000695  0.000036      10        1   5.305768
1      0.000867  0.000053      11        1   5.006436
7      0.000659  0.000033      12        1   4.909324
0      0.000158 -0.000013      13        1   4.574420
12     0.000209 -0.000009      14        1   4.394072
5      0.000167 -0.000012      15        1   4.322232, topic_info=                     Term        Freq       Total Category  logprob  loglift
30           every time i  445.000000  445.000000  Default  30.0000  30.0000
38             time i try  251.000000  251.000000  Default  29.0000  29.0000
4394    since last update  121.000000  121.000000  Default  28.0000  28.0000
1546        even though i  151.000000  151.000000  Default  27.0000  27.0000
5837         i would like  103.000000  103.000000  Default  26.0000  26.0000
...                   ...         ...         ...      ...      ...      ...
16231      hope get fixed    1.816259    2.841376  Topic15  -8.5170   2.6939
4025   i highly recommend    1.718845    2.743963  Topic15  -8.5722   2.6736
20451   doesn’t work keep    1.613108    2.638225  Topic15  -8.6357   2.6495
8120     couple month ago    1.561444    2.586561  Topic15  -8.6682   2.6367
13366    cannot even open    1.484727    2.509844  Topic15  -8.7186   2.6164

[482 rows x 6 columns], token_table=       Topic      Freq                     Term
term                                           
2909      14  0.761821            1 star review
6272       2  0.927631               15 pro max
9869      10  0.835604                2 day ago
336        2  0.957953  2 factor authentication
44         5  0.993321      2 step verification
...      ...       ...                      ...
5159       3  0.920602          would nice able
14174      6  0.890973           would nice see
6626       6  0.865885        would really like
17823      9  0.586422              zero star i
17823     13  0.195474              zero star i

[454 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 9, 14, 7, 5, 10, 4, 11, 3, 15, 2, 8, 1, 13, 6])

In [ ]:
# initialize a tf-idf vectorizer with english stop words
vectorizer_tfidf = TfidfVectorizer(stop_words='english')

# fit and transform the lemmatized words through the vectorizer
vectors_tfidf = vectorizer_tfidf.fit_transform(mental_comments['lemm'])

In [ ]:



n_topics = 6

# initialize a Non-Negative Matrix Factorization (NMF) model
# set random_state=1 to get the same results everytime
# we must also define the number of topics as n_components
clf = decomposition.NMF(n_components=n_topics,random_state=1)

# fit the model using the tf-idf vectors
# get the Weights
W1 = clf.fit_transform(vectors_tfidf)

print('W1 shape (# documents/comments, # topics) is ', W1.shape)

# get the arrays of words that define topics
# get the Hidden Variables
H1 = clf.components_

print('H1 shape (# topics, # terms) is ', H1.shape)

# get the list of all words (a.k.a. terms) found in the tf-idf vectors
vocab = np.array(vectorizer_tfidf.get_feature_names())

# count the number of comments that belong to each topic (primairly)
# should have a dictionary of key:topic, value: #comments belonging to this topic
topics_frequency = collections.Counter(np.array(W1.argsort()[:,-1]))

# order this dictionary and turn it into tuples instead of key, value pairs
# so we shoud should 6 (topic, #comments on this topic) tuples ordered by topic
topics_frequency = collections.OrderedDict(sorted(topics_frequency.items(),key= lambda kv: kv[1],reverse=True))

# total number of comments should be the same as number of rows in the dataset (i.e. mental_comments.shape[0])
total_comments_count = sum(topics_frequency.values())

n_terms_to_display = 10
print('Top ', n_terms_to_display, ' terms for the top',n_topics, 'topics:')
for topic_i,n_comments_per_topic in topics_frequency.items():
  print('Topic {}:{}'.format(
    topic_i+1,
    ", ".join([str(x) for x in vocab[(-H1[topic_i]).argsort(axis=-1)[:n_terms_to_display]]])
  ))